In [84]:
import torch
import pandas as pd
import numpy as np

In [85]:
dataset_name = 'pubmed'
N = 5

In [86]:
clients = {i: torch.load(f"../data/{dataset_name}/{i}_clients.pt") for i in ([-1] + list(range(1, N + 1)))}
clients

{-1: Data(x=[19717, 500], edge_index=[2, 71182], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], index_orig=[19717], map_to_clients=[19717]),
 1: Data(x=[3943, 500], edge_index=[2, 3780], y=[3943], train_mask=[3943], val_mask=[3943], test_mask=[3943], index_orig=[3943]),
 2: Data(x=[3944, 500], edge_index=[2, 3396], y=[3944], train_mask=[3944], val_mask=[3944], test_mask=[3944], index_orig=[3944]),
 3: Data(x=[3943, 500], edge_index=[2, 3404], y=[3943], train_mask=[3943], val_mask=[3943], test_mask=[3943], index_orig=[3943]),
 4: Data(x=[3944, 500], edge_index=[2, 3410], y=[3944], train_mask=[3944], val_mask=[3944], test_mask=[3944], index_orig=[3944]),
 5: Data(x=[3943, 500], edge_index=[2, 3476], y=[3943], train_mask=[3943], val_mask=[3943], test_mask=[3943], index_orig=[3943])}

In [87]:
data = pd.DataFrame(columns=['src_client', 'src_index', 'tgt_client', 'tgt_index'])

# intra client
for client_id in range(1, N + 1):
    local_id_2_global_id = clients[client_id].index_orig
    src, tgt = clients[client_id].edge_index
    client_data = {
        'src_index': local_id_2_global_id[src].tolist(),
        'tgt_index': local_id_2_global_id[tgt].tolist(),
        'src_client': [client_id] * len(src),
        'tgt_client': [client_id] * len(tgt),
    }
    data = pd.concat([data, pd.DataFrame(client_data)], ignore_index=True)

# cross client
id_2_client = dict(clients[-1].map_to_clients)
src, tgt = clients[-1].edge_index
cross_client_data = pd.DataFrame({
    'src_index': src,
    'tgt_index': tgt,
    'src_client': [id_2_client[int(x)] + 1 for x in src],
    'tgt_client': [id_2_client[int(x)] + 1 for x in tgt],
})
data = pd.concat([data, cross_client_data], ignore_index=True)

data

,src_client,src_index,tgt_client,tgt_index
0,1,8193,1,3180
1,1,8193,1,3796
2,1,16387,1,6149
3,1,16387,1,10592
4,1,6,1,3150
...,...,...,...,...
88643,4,19710,1,2304
88644,4,19713,5,12204
88645,1,19714,2,12278
88646,2,19715,3,4284


In [88]:
s = 0.0
for client_id in range(1, N + 1):
    s += len(pd.concat([
        cross_client_data[cross_client_data['src_client'] == client_id]['src_index'].drop_duplicates(),
        cross_client_data[cross_client_data['tgt_client'] == client_id]['tgt_index'].drop_duplicates()
    ]).drop_duplicates()) / clients[client_id].index_orig.size(0)
s / 5

0.9031291106173726

In [89]:
# data.to_csv(f'../data/{dataset_name}/graph.csv', index=False)